# **Install requirement libraries**

In [1]:
!pip install obsei
!pip install transformers
!pip install dateparser
!pip install sentencepiece
!pip install trafilatura
!pip install langdetect

  Using cached obsei-0.0.15-py3-none-any.whl.metadata (44 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached dateparser-1.2.0-py2.py3-none-any.whl.metadata (28 kB)
     ---------------------------------------- 0.0/84.4 kB ? eta -:--:--
     -------------- ------------------------- 30.7/84.4 kB 1.4 MB/s eta 0:00:01
     --------------------------- ---------- 61.4/84.4 kB 825.8 kB/s eta 0:00:01
     -------------------------------------- 84.4/84.4 kB 951.4 kB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 1.0 MB/s eta 0:00:00
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-win_amd64.whl.metad

# Config google news

In [ ]:
# import dateparser
# from gnews import GNews
# from pydantic import PrivateAttr
# from datetime import datetime, date, timedelta, time, timezone
# from typing import Any, Dict, List, Optional
# from urllib import parse
# from obsei.payload import TextPayload
# from obsei.misc.utils import DATETIME_STRING_PATTERN, convert_utc_time
# from obsei.source.base_source import BaseSource, BaseSourceConfig
# from obsei.source.website_crawler_source import (
#     BaseCrawlerConfig,
#     TrafilaturaCrawlerConfig,
# )

# GOOGLE_DATE_TIME_QUERY_PATTERN = "%Y-%m-%d"

# class GoogleNewsConfig(BaseSourceConfig):
#     _google_news_client: GNews = PrivateAttr()
#     TYPE: str = "GoogleNews"
#     query: str
#     country: Optional[str] = "US"
#     language: Optional[str] = "en"
#     max_results: Optional[int] = 100
#     lookup_period: Optional[str] = None
#     after_date: Optional[str] = None
#     before_date: Optional[str] = None
#     fetch_article: Optional[bool] = True
#     crawler_config: Optional[BaseCrawlerConfig] = None

#     def __init__(self, **data: Any):
#         super().__init__(**data)

#         if self.lookup_period and self.after_date:
#             raise AttributeError("Can't use `lookup_period` and `after_date` both")
#         elif not self.after_date and self.before_date:
#             raise AttributeError("Can't use `before_date` without `after_date` or `lookup_period`")

#         if self.lookup_period:
#             after_time = convert_utc_time(self.lookup_period)
#             self.after_date = after_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)

#         if not self.before_date:
#             before_time = datetime.combine(date.today(), time(tzinfo=timezone.utc)) + timedelta(days=1)
#             self.before_date = before_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)

#         self._google_news_client = GNews(
#             language=self.language,
#             country=self.country,
#             max_results=self.max_results,
#         )

#         if not self.crawler_config:
#             self.crawler_config = TrafilaturaCrawlerConfig(urls=[])

#     def get_client(self) -> GNews:
#         return self._google_news_client


# class GoogleNewsSource(BaseSource):
#     NAME: Optional[str] = "GoogleNews"

#     def lookup(self, config: GoogleNewsConfig, **kwargs) -> List[TextPayload]:  # type: ignore[override]
#         source_responses: List[TextPayload] = []

#         # Get data from state
#         id: str = kwargs.get("id", None)
#         state: Optional[Dict[str, Any]] = (
#             None
#             if id is None or self.store is None
#             else self.store.get_source_state(id)
#         )
#         update_state: bool = True if id else False
#         state = state or dict()
#         lookup_period: str = state.get("since_time", None)
#         since_time = None if not lookup_period else convert_utc_time(lookup_period)
#         last_since_time = since_time

#         last_after_time = convert_utc_time(config.after_date) if config.after_date else None
#         if since_time and last_after_time:
#             last_after_time = since_time if since_time > last_after_time else last_since_time
#         elif not last_after_time:
#             last_after_time = datetime.combine(date.today(), time(tzinfo=timezone.utc))

#         before_time = convert_utc_time(config.before_date) if config.after_date else None
#         if not before_time or before_time > datetime.combine(date.today(), time(tzinfo=timezone.utc)):
#             before_time = datetime.combine(date.today(), time(tzinfo=timezone.utc)) + timedelta(days=1)

#         google_news_client = config.get_client()
#         more_data_exist = True
#         while more_data_exist and before_time > last_after_time:
#             after_time = before_time - timedelta(days=1)
#             after_date = after_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)
#             before_date = before_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)

#             new_query = f'{config.query}+after:{after_date}+before:{before_date}'
#             query = parse.quote(new_query, errors='ignore')

#             before_time = after_time

#             articles = google_news_client.get_news(query)

#             for article in articles:
#                 published_date = (
#                     None
#                     if article["published date"] == ""
#                     else dateparser.parse(article["published date"])
#                 )

#                 if config.fetch_article and config.crawler_config:
#                     extracted_data = config.crawler_config.extract_url(url=article["url"])

#                     if extracted_data is not None and extracted_data.get("text", None) is not None:
#                         article_text = extracted_data["text"]
#                         del extracted_data["text"]
#                     else:
#                         article_text = ""

#                     article["extracted_data"] = extracted_data
#                 else:
#                     article_text = article["description"]

#                 source_responses.append(
#                     TextPayload(
#                         processed_text=f"{article['title']}.\n\n {article_text}",
#                         meta=vars(article) if hasattr(article, "__dict__") else article,
#                         source_name=self.NAME,
#                     )
#                 )

#                 if config.max_results is not None and len(source_responses) >= config.max_results:
#                     source_responses = source_responses[:config.max_results]
#                     more_data_exist = False
#                     break

#                 if published_date and since_time and published_date < since_time:
#                     more_data_exist = False
#                     break
#                 if last_since_time is None or (
#                     published_date and last_since_time < published_date
#                 ):
#                     last_since_time = published_date

#             if update_state and last_since_time and self.store is not None:
#                 state["since_time"] = last_since_time.strftime(DATETIME_STRING_PATTERN)
#                 self.store.update_source_state(workflow_id=id, state=state)

#         return source_responses

# Crawl gg news

In [ ]:
# import pandas as pd
# query="seagames"
# max_results=5
# lookup_period="3Y"
# language="vi"
# country="VN"
# userName = []
# content = []
# rating = []
# id = []
# title = []
# platform = []
# extracted_data=[]
# source_config = GoogleNewsConfig(
# query=query,
# max_results=max_results,
# lookup_period=lookup_period,
# language=language,
# country=country,
# )
# source = GoogleNewsSource()
# data = source.lookup(source_config)
# #print(data)
# for i in range(0, max_results):
#     data[i]=dict(data[i])
#     #print(data[i])
#     id.append(None)
#     rating.append(None)
#     platform.append('Google News')
#     extracted_data.append(data[i]["meta"]["extracted_data"])
#     Not_none_values = filter(None.__ne__, extracted_data)
#     extracted_data = list(Not_none_values)
#     for i in range(len(extracted_data)):
#       extracted_data[i]=dict(extracted_data[i])
#       print(extracted_data[i])
#       userName.append(extracted_data[i]["author"])
#       title.append(extracted_data[i]["title"])
#       content.append(extracted_data[i]["raw_text"])
# data=pd.DataFrame(list(zip(title,userName,content)),columns=["title","author","content"])


ERROR:trafilatura.xml:empty link: None {'target': '/lich-thi-dau/lich-thi-daucau-longsea-games-31-hom-nay-vtv6-vtv5-truc-tiep-n20220517085553236.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/bong-da-viet/ket-quabong-da-sea-games-31ket-qua-bong-dau23-viet-nam-moi-nhat-n20220514171621213.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/sea-games-31/truc-tiep-sea-games-31-hom-nay-215-nu-viet-nam-tranh-hcv-voi-thai-lan-n20220521113223135.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/sea-games-31/truc-tiep-sea-games-145-vtv6-vtv5-dien-kinh-boi-loi-bong-chuyen-esports-wushu-n20220513203805963.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/truc-tiep-bong-da/vtv6-truc-tiep-bong-da-nu-viet-nam-vs-thai-lan-chung-ket-sea-games-31-19h00-215-n20220520115639702.htm'}


{'title': 'Lịch thi đấu và trực tiếp cầu lông SEA Games 31 (VTV6, VTV5)', 'author': 'PV; BTV-TTVH', 'hostname': 'thethaovanhoa.vn', 'date': '2022-05-19', 'categories': 'Thể thao', 'tags': 'Lich thi dau Sea games 31, lich thi dau sea games, trực tiếp cầu lông Sea games 31, xem trực tiếp cầu lông sea games, lich thi dau cau long, trực tiếp cầu lông việt nam;Lich thi dau Sea games 31;lich thi dau sea games;trực tiếp cầu lông Sea games 31;xem trực tiếp cầu lông sea games;lich thi dau cau long;trực tiếp cầu lông việt nam', 'fingerprint': 'j+u6ya16H0vdxcOe4epzsKjBhXE=', 'id': '91ec24fb', 'license': None, 'comments': None, 'raw_text': 'Lịch thi đấu và trực tiếp cầu lông SEA Games 31 (VTV6, VTV5) Thứ Năm, 19/05/2022 10:55 GMT+7 Lịch thi đấu và trực tiếp cầu lông SEA Games 31 - Xem trực tiếp cầu lông SEA Games 31 năm 2022 tại Việt Nam. Lịch thi đấu môn cầu lông SEA Games 31. Lịch thi đấu cầu lông SEA Games 31 - Xem VTV6 VTV5 trực tiếp cầu lông SEA Games 31 năm 2022. Lịch thi đấu môn cầu lông Vi

In [ ]:
data=data.drop_duplicates()

In [ ]:
data

,title,author,content
0,Đảm bảo mức cao nhất trong công tác y tế phục ...,Lê Hảo TTXVN; Vietnam,Đảm bảo mức cao nhất trong công tác y tế phục ...
2,"IPC không cho phép Nga, Belarus tham gia Paral...",Thúc Anh TTXVN; Vietnam,"IPC không cho phép Nga, Belarus tham gia Paral..."
5,U23 Đông Nam Á 2022: Tuyển Việt Nam bắt đầu hà...,Minh Tiến TTXVN; Vietnam,U23 Đông Nam Á 2022: Tuyển Việt Nam bắt đầu hà...
9,Phát hiện trên 27.000 trường hợp vi phạm nồng ...,Chu Thanh Vân TTXVN; Vietnam,Phát hiện trên 27.000 trường hợp vi phạm nồng ...


# config website

In [1]:
import json
import logging
from abc import abstractmethod
from typing import List, Optional

import mmh3

from obsei.payload import TextPayload
from obsei.source.base_source import BaseSource, BaseSourceConfig

logger = logging.getLogger(__name__)


class BaseCrawlerConfig(BaseSourceConfig):
    TYPE: str = "BaseCrawler"

    @abstractmethod
    def extract_url(self, url: str, url_id: str = None):
        pass

    @abstractmethod
    def find_urls(self, url: str):
        pass


class TrafilaturaCrawlerConfig(BaseCrawlerConfig):
    # To understand about these configuration params refer:
    # https://trafilatura.readthedocs.io/
    _output_format: str = "json"
    TYPE: str = "Crawler"
    urls: List[str]
    include_comments: bool = False
    include_tables: bool = True
    no_fallback: bool = False
    include_images: bool = False
    include_formatting: bool = False
    deduplicate: bool = True
    no_ssl: bool = False
    is_feed: bool = False
    is_sitemap: bool = False
    include_links: bool = True
    target_language: Optional[str] = None
    url_blacklist: Optional[List[str]] = None

    def extract_url(self, url: str, url_id: str = None):
        try:
            from trafilatura import extract, fetch_url
        except:
            logger.error("Trafilatura is not installed, install as follows: pip install trafilatura")
            return []

        url_id = url_id or "{:02x}".format(mmh3.hash(url, signed=False))
        url_content = fetch_url(
            url=url,
            no_ssl=self.no_ssl,
        )
        extracted_dict = None
        if url_content is not None:
            extracted_data = extract(
                filecontent=url_content,
                record_id=url_id,
                no_fallback=self.no_fallback,
                output_format=self._output_format,
                include_comments=self.include_comments,
                include_tables=self.include_tables,
                include_images=self.include_images,
                include_formatting=self.include_formatting,
                include_links=self.include_links,
                deduplicate=self.deduplicate,
                url_blacklist=self.url_blacklist,
                target_language=self.target_language,
            )

            if extracted_data:
                extracted_dict = json.loads(extracted_data)
                if "raw-text" in extracted_dict:
                    del extracted_dict["raw-text"]

        return extracted_dict

    def find_urls(self, url: str):
        try:
            from trafilatura import feeds, sitemaps
        except:
            logger.error("Trafilatura is not installed, install as follows: pip install trafilatura")
            return []

        urls: List[str] = []
        if self.is_sitemap:
            urls = sitemaps.sitemap_search(url=url, target_lang=self.target_language)
        elif self.is_feed:
            urls = feeds.find_feed_urls(url=url, target_lang=self.target_language)

        return urls


class TrafilaturaCrawlerSource(BaseSource):
    NAME: Optional[str] = "Crawler"

    def lookup(  # type: ignore[override]
        self, config: TrafilaturaCrawlerConfig, **kwargs
    ) -> List[TextPayload]:
        source_responses: List[TextPayload] = []

        final_urls = []
        if config.is_sitemap or config.is_feed:
            for url in config.urls:
                final_urls.extend(config.find_urls(url=url))
        else:
            final_urls = config.urls

        for url in final_urls:
            extracted_data = config.extract_url(url=url)
            if extracted_data is None:
                logger.warning(f"Unable to crawl {url}, hence skipping it")
                continue
            comments = (
                "" if "comments" not in extracted_data else extracted_data["comments"]
            )
            source_responses.append(
                TextPayload(
                    processed_text=f"{extracted_data['text']}. {comments}",
                    meta=extracted_data,
                    source_name=self.NAME,
                )
            )

        return source_responses

03/09/2024 10:17:42 - WARNING - obsei -   
By default `pip install obsei` will only install core dependencies.
To install all required dependencies use `pip install obsei[all]`.
Refer https://obsei.com/#install-obsei for more options.



# Crawl website

In [2]:
# from obsei.source.website_crawler_source import (
#     TrafilaturaCrawlerConfig,
#     TrafilaturaCrawlerSource,
# )
import pandas as pd
#Single URL
from trafilatura import extract, fetch_url
source_config=TrafilaturaCrawlerConfig(
    urls=["https://www.booking.com/hotel/vn/mondial-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=caf7da4c016da58147ec5079d2cf53fd&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;hpos=1;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709356931;srpvid=afb325be82f0025a;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/hue-nino.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709908390&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/jade-scene.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=3;hpos=3;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908749;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/na-na-homestay-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;hpos=4;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908787;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/myfa-house.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709908809&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/golden-star-villa.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=6;hpos=6;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908828;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/huenam-residence.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=7;hpos=7;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908850;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/khach-san-le-domaine-de-cocodo.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=8;hpos=8;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908868;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/gold-2.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=9;hpos=9;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908885;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/sala-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=10;hpos=10;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908901;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/thien-phu-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=11;hpos=11;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908918;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/soleil-boutique-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=12;hpos=12;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908941;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lala-homestay-hue1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=13;hpos=13;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909079;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lemi-thanh-pho-hue1.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=14&hapos=14&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709909098&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/khach-san-thanh-lich.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=15&hapos=15&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709909112&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/nam-gold-thanh-pho-hue1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=16;hpos=16;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909133;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/familia.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=17;hpos=17;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909150;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/la-perle.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=18;hpos=18;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909165;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/sunny-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=19;hpos=19;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909180;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/melia-vinpearl-hue-hotel.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=20;hpos=20;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909205;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/duc-loc-2.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=21;hpos=21;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909221;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/tung-hostel-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=22;hpos=22;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909241;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/orchid-reverside-villa.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=23;hpos=23;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909262;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/shine-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=24;hpos=24;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909284;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/robin-house.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=25;hpos=25;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909297;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lis-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1165521502_389213447_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=1165521502_389213447_2_0_0;hpos=1;matching_block_id=1165521502_389213447_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1165521502_389213447_2_0_0__25500000;srepoch=1709953637;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/mayhome-deluxe-45m2-balcony-5-min-to-my-khe-beach.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1145947804_387459805_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=2;highlighted_blocks=1145947804_387459805_2_0_0;hpos=2;matching_block_id=1145947804_387459805_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1145947804_387459805_2_0_0__31450000;srepoch=1709953764;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/masion-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1141633703_387009930_0_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=3;highlighted_blocks=1141633703_387009930_0_0_0;hpos=3;matching_block_id=1141633703_387009930_0_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1141633703_387009930_0_0_0__31185000;srepoch=1709953778;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/royal-family.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=123994605_121940906_2_2_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;highlighted_blocks=123994605_121940906_2_2_0;hpos=4;matching_block_id=123994605_121940906_2_2_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=123994605_121940906_2_2_0__44640000;srepoch=1709953842;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/anh-dao-amp-apartment.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1083315401_381604651_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=5;highlighted_blocks=1083315401_381604651_2_0_0;hpos=5;matching_block_id=1083315401_381604651_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1083315401_381604651_2_0_0__32580000;srepoch=1709953863;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/golden-sun-apartment-by-new-sky-travel.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1149297704_388955202_2_2_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=6;highlighted_blocks=1149297704_388955202_2_2_0;hpos=6;matching_block_id=1149297704_388955202_2_2_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1149297704_388955202_2_2_0__32400000;srepoch=1709953879;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/hai-trieu-da-nang1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=845306512_351498422_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=7;highlighted_blocks=845306512_351498422_2_0_0;hpos=7;matching_block_id=845306512_351498422_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=845306512_351498422_2_0_0__38880000;srepoch=1709953894;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/ngoc-bich-amp-apartment.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=933900207_367355760_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=8;highlighted_blocks=933900207_367355760_2_0_0;hpos=8;matching_block_id=933900207_367355760_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=933900207_367355760_2_0_0__32500000;srepoch=1709953907;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/tung-phuong.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=111269617_125360138_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=9;highlighted_blocks=111269617_125360138_2_0_0;hpos=9;matching_block_id=111269617_125360138_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=111269617_125360138_2_0_0__30500000;srepoch=1709953926;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/quoc-cuong-ii-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=40204108_125357763_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=10;highlighted_blocks=40204108_125357763_2_0_0;hpos=10;matching_block_id=40204108_125357763_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=40204108_125357763_2_0_0__33858000;srepoch=1709953941;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/cordial.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=442399116_0_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=11;highlighted_blocks=442399116_0_2_0_0;hpos=11;matching_block_id=442399116_0_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=442399116_0_2_0_0__40686609;srepoch=1709953963;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/duy-tung-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=600513708_235472482_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=12;highlighted_blocks=600513708_235472482_2_0_0;hpos=12;matching_block_id=600513708_235472482_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=600513708_235472482_2_0_0__24950000;srepoch=1709953979;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/phu-an-hai-chau.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1086666301_382370093_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=13;highlighted_blocks=1086666301_382370093_2_0_0;hpos=13;matching_block_id=1086666301_382370093_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1086666301_382370093_2_0_0__33536600;srepoch=1709953995;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/golden-house-son-tra.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1152252504_388321446_0_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=14;highlighted_blocks=1152252504_388321446_0_0_0;hpos=14;matching_block_id=1152252504_388321446_0_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1152252504_388321446_0_0_0__22000000;srepoch=1709954016;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/quang-minh-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1020596003_374445827_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=15;highlighted_blocks=1020596003_374445827_2_0_0;hpos=15;matching_block_id=1020596003_374445827_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1020596003_374445827_2_0_0__44460000;srepoch=1709954031;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/khach-san-goldtime.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=851567802_352455286_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=16;highlighted_blocks=851567802_352455286_2_0_0;hpos=16;matching_block_id=851567802_352455286_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=851567802_352455286_2_0_0__28500000;srepoch=1709954044;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/meet-banh-mi-and-coffee.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1100925901_382929778_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=17;highlighted_blocks=1100925901_382929778_2_0_0;hpos=17;matching_block_id=1100925901_382929778_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1100925901_382929778_2_0_0__35781700;srepoch=1709954060;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/ibiza-riverfront.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=0_0_2_1_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=18;highlighted_blocks=0_0_2_1_0;hpos=18;matching_block_id=0_0_2_1_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=0_0_2_1_0__45624305;srepoch=1709954075;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/open.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=176871818_210492506_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=19;highlighted_blocks=176871818_210492506_2_0_0;hpos=19;matching_block_id=176871818_210492506_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=176871818_210492506_2_0_0__34080000;srepoch=1709954090;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/ruby-light-danang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=371573519_125380885_0_1_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=20;highlighted_blocks=371573519_125380885_0_1_0;hpos=20;matching_block_id=371573519_125380885_0_1_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=371573519_125380885_0_1_0__45900000;srepoch=1709954106;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/song-cong-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=90649120_86263100_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=21;highlighted_blocks=90649120_86263100_2_0_0;hpos=21;matching_block_id=90649120_86263100_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=90649120_86263100_2_0_0__45000000;srepoch=1709954131;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/happy-day.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=59399602_121938054_0_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=22;highlighted_blocks=59399602_121938054_0_0_0;hpos=22;matching_block_id=59399602_121938054_0_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=59399602_121938054_0_0_0__36900000;srepoch=1709954147;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lan-nhi.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=132401111_272066168_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=23;highlighted_blocks=132401111_272066168_2_0_0;hpos=23;matching_block_id=132401111_272066168_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=132401111_272066168_2_0_0__30600000;srepoch=1709954164;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/bona-hostel-son-tra1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1092328501_382107714_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=24;highlighted_blocks=1092328501_382107714_2_0_0;hpos=24;matching_block_id=1092328501_382107714_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1092328501_382107714_2_0_0__26792900;srepoch=1709954179;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/dana-house-elc.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1099170901_384059532_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=25;highlighted_blocks=1099170901_384059532_2_0_0;hpos=25;matching_block_id=1099170901_384059532_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1099170901_384059532_2_0_0__60000000;srepoch=1709954198;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl"],
    include_comments = False,
    # include_tables = False,
    # no_fallback = False,
    # include_images = False,
    # include_formatting = False,
    # deduplicate = False,
    # no_ssl = False,
    # is_feed = False,
    #is_sitemap = True,
    # include_links = False,
)
source=TrafilaturaCrawlerSource()
urls=["https://www.booking.com/hotel/vn/mondial-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=caf7da4c016da58147ec5079d2cf53fd&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;hpos=1;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709356931;srpvid=afb325be82f0025a;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/hue-nino.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709908390&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/jade-scene.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=3;hpos=3;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908749;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/na-na-homestay-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;hpos=4;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908787;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/myfa-house.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709908809&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/golden-star-villa.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=6;hpos=6;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908828;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/huenam-residence.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=7;hpos=7;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908850;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/khach-san-le-domaine-de-cocodo.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=8;hpos=8;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908868;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/gold-2.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=9;hpos=9;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908885;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/sala-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=10;hpos=10;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908901;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/thien-phu-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=11;hpos=11;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908918;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/soleil-boutique-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=12;hpos=12;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709908941;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lala-homestay-hue1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=13;hpos=13;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909079;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lemi-thanh-pho-hue1.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=14&hapos=14&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709909098&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/khach-san-thanh-lich.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&aid=2311236&ucfs=1&arphpl=1&dest_id=-3715887&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=15&hapos=15&sr_order=popularity&srpvid=6e0e5b4da90901d8&srepoch=1709909112&from_sustainable_property_sr=1&from=searchresults#hotelTmpl",
      "https://www.booking.com/hotel/vn/nam-gold-thanh-pho-hue1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=16;hpos=16;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909133;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/familia.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=17;hpos=17;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909150;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/la-perle.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=18;hpos=18;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909165;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/sunny-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=19;hpos=19;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909180;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/melia-vinpearl-hue-hotel.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=20;hpos=20;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909205;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/duc-loc-2.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=21;hpos=21;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909221;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/tung-hostel-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=22;hpos=22;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909241;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/orchid-reverside-villa.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=23;hpos=23;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909262;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/shine-thanh-pho-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=24;hpos=24;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909284;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/robin-house.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=25;hpos=25;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1709909297;srpvid=6e0e5b4da90901d8;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lis-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1165521502_389213447_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=1165521502_389213447_2_0_0;hpos=1;matching_block_id=1165521502_389213447_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1165521502_389213447_2_0_0__25500000;srepoch=1709953637;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/mayhome-deluxe-45m2-balcony-5-min-to-my-khe-beach.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1145947804_387459805_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=2;highlighted_blocks=1145947804_387459805_2_0_0;hpos=2;matching_block_id=1145947804_387459805_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1145947804_387459805_2_0_0__31450000;srepoch=1709953764;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/masion-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1141633703_387009930_0_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=3;highlighted_blocks=1141633703_387009930_0_0_0;hpos=3;matching_block_id=1141633703_387009930_0_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1141633703_387009930_0_0_0__31185000;srepoch=1709953778;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/royal-family.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=123994605_121940906_2_2_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;highlighted_blocks=123994605_121940906_2_2_0;hpos=4;matching_block_id=123994605_121940906_2_2_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=123994605_121940906_2_2_0__44640000;srepoch=1709953842;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/anh-dao-amp-apartment.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1083315401_381604651_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=5;highlighted_blocks=1083315401_381604651_2_0_0;hpos=5;matching_block_id=1083315401_381604651_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1083315401_381604651_2_0_0__32580000;srepoch=1709953863;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/golden-sun-apartment-by-new-sky-travel.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1149297704_388955202_2_2_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=6;highlighted_blocks=1149297704_388955202_2_2_0;hpos=6;matching_block_id=1149297704_388955202_2_2_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1149297704_388955202_2_2_0__32400000;srepoch=1709953879;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/hai-trieu-da-nang1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=845306512_351498422_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=7;highlighted_blocks=845306512_351498422_2_0_0;hpos=7;matching_block_id=845306512_351498422_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=845306512_351498422_2_0_0__38880000;srepoch=1709953894;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/ngoc-bich-amp-apartment.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=933900207_367355760_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=8;highlighted_blocks=933900207_367355760_2_0_0;hpos=8;matching_block_id=933900207_367355760_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=933900207_367355760_2_0_0__32500000;srepoch=1709953907;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/tung-phuong.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=111269617_125360138_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=9;highlighted_blocks=111269617_125360138_2_0_0;hpos=9;matching_block_id=111269617_125360138_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=111269617_125360138_2_0_0__30500000;srepoch=1709953926;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/quoc-cuong-ii-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=40204108_125357763_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=10;highlighted_blocks=40204108_125357763_2_0_0;hpos=10;matching_block_id=40204108_125357763_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=40204108_125357763_2_0_0__33858000;srepoch=1709953941;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/cordial.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=442399116_0_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=11;highlighted_blocks=442399116_0_2_0_0;hpos=11;matching_block_id=442399116_0_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=442399116_0_2_0_0__40686609;srepoch=1709953963;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/duy-tung-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=600513708_235472482_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=12;highlighted_blocks=600513708_235472482_2_0_0;hpos=12;matching_block_id=600513708_235472482_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=600513708_235472482_2_0_0__24950000;srepoch=1709953979;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/phu-an-hai-chau.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1086666301_382370093_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=13;highlighted_blocks=1086666301_382370093_2_0_0;hpos=13;matching_block_id=1086666301_382370093_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1086666301_382370093_2_0_0__33536600;srepoch=1709953995;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/golden-house-son-tra.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1152252504_388321446_0_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=14;highlighted_blocks=1152252504_388321446_0_0_0;hpos=14;matching_block_id=1152252504_388321446_0_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1152252504_388321446_0_0_0__22000000;srepoch=1709954016;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/quang-minh-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1020596003_374445827_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=15;highlighted_blocks=1020596003_374445827_2_0_0;hpos=15;matching_block_id=1020596003_374445827_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1020596003_374445827_2_0_0__44460000;srepoch=1709954031;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/khach-san-goldtime.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=851567802_352455286_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=16;highlighted_blocks=851567802_352455286_2_0_0;hpos=16;matching_block_id=851567802_352455286_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=851567802_352455286_2_0_0__28500000;srepoch=1709954044;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/meet-banh-mi-and-coffee.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1100925901_382929778_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=17;highlighted_blocks=1100925901_382929778_2_0_0;hpos=17;matching_block_id=1100925901_382929778_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1100925901_382929778_2_0_0__35781700;srepoch=1709954060;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/ibiza-riverfront.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=0_0_2_1_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=18;highlighted_blocks=0_0_2_1_0;hpos=18;matching_block_id=0_0_2_1_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=0_0_2_1_0__45624305;srepoch=1709954075;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/open.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=176871818_210492506_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=19;highlighted_blocks=176871818_210492506_2_0_0;hpos=19;matching_block_id=176871818_210492506_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=176871818_210492506_2_0_0__34080000;srepoch=1709954090;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/ruby-light-danang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=371573519_125380885_0_1_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=20;highlighted_blocks=371573519_125380885_0_1_0;hpos=20;matching_block_id=371573519_125380885_0_1_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=371573519_125380885_0_1_0__45900000;srepoch=1709954106;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/song-cong-da-nang.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=90649120_86263100_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=21;highlighted_blocks=90649120_86263100_2_0_0;hpos=21;matching_block_id=90649120_86263100_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=90649120_86263100_2_0_0__45000000;srepoch=1709954131;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/happy-day.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=59399602_121938054_0_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=22;highlighted_blocks=59399602_121938054_0_0_0;hpos=22;matching_block_id=59399602_121938054_0_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=59399602_121938054_0_0_0__36900000;srepoch=1709954147;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/lan-nhi.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=132401111_272066168_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=23;highlighted_blocks=132401111_272066168_2_0_0;hpos=23;matching_block_id=132401111_272066168_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=132401111_272066168_2_0_0__30600000;srepoch=1709954164;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/bona-hostel-son-tra1.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1092328501_382107714_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=24;highlighted_blocks=1092328501_382107714_2_0_0;hpos=24;matching_block_id=1092328501_382107714_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1092328501_382107714_2_0_0__26792900;srepoch=1709954179;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl",
      "https://www.booking.com/hotel/vn/dana-house-elc.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=a9624d24bcced274d7f7cdf482512d9e&all_sr_blocks=1099170901_384059532_2_0_0;checkin=2024-03-09;checkout=2024-03-10;dest_id=-3712125;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=25;highlighted_blocks=1099170901_384059532_2_0_0;hpos=25;matching_block_id=1099170901_384059532_2_0_0;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=1099170901_384059532_2_0_0__60000000;srepoch=1709954198;srpvid=f06715ef220300e0;type=total;ucfs=1&#hotelTmpl"]
source_response_list=source.lookup(source_config)
print(source_response_list)
#Extract information from data
id = []
hostname = []
title = []
raw_text = []
rating = []
platform = []
for i in range(0, len(urls)):
    source_response_list[i] = dict(source_response_list[i])
    id.append(source_response_list[i]["meta"]["id"])
    hostname.append(source_response_list[i]["meta"]["hostname"])
    title.append(source_response_list[i]["meta"]["title"])
    raw_text.append(source_response_list[i]["meta"]["raw_text"])
data = pd.DataFrame(list(zip(id, hostname, title, raw_text)), columns=['id','author','title','content'])

C:\Users\ThongHuynh\AppData\Local\Temp\ipykernel_15548\3781519994.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
03/09/2024 10:17:50 - INFO - urllib3.poolmanager -   Redirecting https://www.booking.com/hotel/vn/mondial-hue.vi.html?aid=2311236&label=vi-vn-booking-desktop-ztDRHDT0cyr%2Allk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1028809%3Ali%3Adec%3Adm&sid=caf7da4c016da58147ec5079d2cf53fd&dest_id=-3715887;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;hpos=1;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=po

[TextPayload(segmented_data={}, meta={'title': '★★★★ Mondial Hotel, Huế, Việt Nam', 'author': None, 'hostname': 'booking.com', 'date': '2017-09-19', 'fingerprint': 'c3bcf295f55e6e17', 'id': '58c06e2', 'license': None, 'comments': None, 'raw_text': 'Mondial Hotel Mondial Hotel  \nVị trí tuyệt vời - Hiển thị bản đồ\n \n\n \n Tiện nghi  8,0  \n Sự sạch sẽ  8,3  \n Thoải mái  8,3  \n Đáng giá tiền  8,2  \n Địa điểm  8,4  \n Tất cả  7,9  \n WiFi miễn phí  7,7  Bữa sáng ăn ngon, ăn thỏa thích. Nằm ngay trung tâm thành phố nên thuận tiện di chuyển  Bạn đủ điều kiện nhận giảm giá Genius tại Mondial Hotel! Để tiết kiệm tại chỗ nghỉ này, bạn chỉ cần  đăng nhập. \n Mondial Hotel Hue cung cấp chỗ ở hiện đại ở trung tâm thành phố, cách Sông Hương thơ mộng một quãng đi bộ. Khách sạn có hồ bơi, 3 lựa chọn ăn uống và quầy bar tại sảnh khách. Wi-Fi miễn phí cũng được cung cấp cho khách. Các phòng và suite lắp máy điều hòa tại Mondial Hotel Hue có tầm nhìn ra thành phố, hồ bơi hoặc núi non. Mỗi phòng đề

In [3]:
data

,id,author,title,content
0,58c06e2,booking.com,"★★★★ Mondial Hotel, Huế, Việt Nam",Mondial Hotel Mondial Hotel \nVị trí tuyệt vờ...
1,a29f7a75,booking.com,"★★ Hue Nino Hotel, Huế, Việt Nam",Hue Nino Hotel Hue Nino Hotel \nVị trí xuất s...
2,48f61801,booking.com,"★★★ Jade Scene Hotel, Huế, Việt Nam",Jade Scene Hotel Jade Scene Hotel \nVị trí xu...
3,bc17f20b,booking.com,"Nana Homestay & Villa, Huế, Việt Nam",Nana Homestay & Villa Nana Homestay & Villa \...
4,9d6dcb48,booking.com,"MyFa House, Huế, Việt Nam",MyFa House MyFa House \nVị trí xuất sắc - hiể...
5,b60a7eb5,booking.com,"★★★ Golden Star Villa Hue, Huế, Việt Nam",Golden Star Villa Hue Golden Star Villa Hue \...
6,b3dd1d62,booking.com,"HueNam Residence, Huế, Việt Nam",HueNam Residence HueNam Residence \nVị trí xu...
7,160a05ee,booking.com,"Khách sạn Le Domaine De Cocodo, Huế, Việt Nam",Khách sạn Le Domaine De Cocodo Khách sạn Le Do...
8,a535dd90,booking.com,"★★★ Thuy Duong Boutique Hotel Hue, Huế, Việt Nam",Thuy Duong Boutique Hotel Hue Thuy Duong Bouti...
9,667c234e,booking.com,"★★★ SALA HOTEL HUE, Huế, Việt Nam",SALA HOTEL HUE SALA HOTEL HUE \nVị trí tuyệt ...


In [ ]:
data["content"]

0    Mondial Hotel Mondial Hotel  \nVị trí tuyệt vờ...
Name: content, dtype: object

In [13]:
import re
for val in data["content"].values:
  text=str(val)
  print(text)

Mondial Hotel Mondial Hotel  
Vị trí tuyệt vời - Hiển thị bản đồ
 

 
 Tiện nghi  8,0  
 Sự sạch sẽ  8,3  
 Thoải mái  8,3  
 Đáng giá tiền  8,2  
 Địa điểm  8,4  
 Tất cả  7,9  
 WiFi miễn phí  7,7  Bữa sáng ăn ngon, ăn thỏa thích. Nằm ngay trung tâm thành phố nên thuận tiện di chuyển  Bạn đủ điều kiện nhận giảm giá Genius tại Mondial Hotel! Để tiết kiệm tại chỗ nghỉ này, bạn chỉ cần  đăng nhập. 
 Mondial Hotel Hue cung cấp chỗ ở hiện đại ở trung tâm thành phố, cách Sông Hương thơ mộng một quãng đi bộ. Khách sạn có hồ bơi, 3 lựa chọn ăn uống và quầy bar tại sảnh khách. Wi-Fi miễn phí cũng được cung cấp cho khách. Các phòng và suite lắp máy điều hòa tại Mondial Hotel Hue có tầm nhìn ra thành phố, hồ bơi hoặc núi non. Mỗi phòng đều được trang bị TV, tủ lạnh mini và tiện nghi pha trà/cà phê Quý khách có thể tận hưởng dịch vụ mát-xa thư giãn tại spa. Các chuyến đi trong ngày và dịch vụ cho thuê xe hơi có thể được thu xếp tại bàn đặt tour. Nhà hàng và Bar Sông Hương phục vụ các món ăn địa 

In [4]:
import pandas as pd
import re

# Giả sử df là DataFrame của bạn và 'content' là cột chứa dữ liệu

# Hàm để tách điểm đánh giá
def tach_diem_danh_gia(content):
    # Tạo một từ điển rỗng để lưu trữ kết quả
    diem_danh_gia = {}
    # Tìm tất cả các điểm số và loại đánh giá bằng regex
    matches = re.findall(r"(\w[\w\s]+\w)\s+(\d+,\d+)", content)
    for match in matches:
        key = match[0].strip()  # Loại đánh giá
        value = match[1].replace(',', '.')  # Điểm đánh giá, chuyển dấu phẩy thành dấu chấm
        diem_danh_gia[key] = float(value)  # Lưu vào từ điển
    return diem_danh_gia

# Áp dụng hàm tách điểm đánh giá lên từng hàng của cột 'content' và tạo một cột mới
data['Điểm đánh giá'] = data['content'].apply(tach_diem_danh_gia)

In [9]:
data['Điểm đánh giá'][1]

{'hiển thị bản đồ\n \n\n \n Tiện nghi': 8.7,
 'Sự sạch sẽ': 9.1,
 'Thoải mái': 9.1,
 'Đáng giá tiền': 9.3,
 'Địa điểm': 9.2,
 'Tất cả': 8.9,
 'WiFi miễn phí': 9.1,
 'họ cho điểm': 9.4,
 'Phòng gia đình \n\n Hue Nino Hotel cách trung tâm Huế': 1.1}

In [5]:
def tach_chinh_sach_huy(content):
    # Sử dụng biểu thức chính quy để tìm kiếm thông tin
    # Giả sử cấu trúc cố định là "Hủy đặt phòng/ Trả trước" theo sau là thông tin cần tìm
    pattern = r"Hủy đặt phòng/ Trả trước\s*(.*?)\s*(?=\n|$)"
    match = re.search(pattern, content, re.DOTALL)
    if match:
        return match.group(1).strip()  # Trả về phần tìm được, loại bỏ khoảng trắng thừa
    else:
        return ""  # Trả về chuỗi rỗng nếu không tìm thấy

# Áp dụng hàm tách thông tin lên từng hàng của cột 'content' và tạo một cột mới
data['Hủy đặt phòng/ Trả trước'] = data['content'].apply(tach_chinh_sach_huy)
data

,id,author,title,content,Điểm đánh giá,Hủy đặt phòng/ Trả trước
0,58c06e2,booking.com,"★★★★ Mondial Hotel, Huế, Việt Nam",Mondial Hotel Mondial Hotel \nVị trí tuyệt vờ...,"{'Hiển thị bản đồ Tiện nghi': 8.0, 'Sự s...",Các chính sách hủy và thanh toán trước có khác...
1,a29f7a75,booking.com,"★★ Hue Nino Hotel, Huế, Việt Nam",Hue Nino Hotel Hue Nino Hotel \nVị trí xuất s...,"{'hiển thị bản đồ Tiện nghi': 8.7, 'Sự s...",Các chính sách hủy và thanh toán trước có khác...
2,48f61801,booking.com,"★★★ Jade Scene Hotel, Huế, Việt Nam",Jade Scene Hotel Jade Scene Hotel \nVị trí xu...,"{'hiển thị bản đồ Tiện nghi': 9.4, 'Sự s...",Các chính sách hủy và thanh toán trước có khác...
3,bc17f20b,booking.com,"Nana Homestay & Villa, Huế, Việt Nam",Nana Homestay & Villa Nana Homestay & Villa \...,"{'Hiển thị bản đồ Tiện nghi': 8.7, 'Sự s...",nhập ngày tháng lưu trú và kiểm tra các điều ...
4,9d6dcb48,booking.com,"MyFa House, Huế, Việt Nam",MyFa House MyFa House \nVị trí xuất sắc - hiể...,"{'hiển thị bản đồ Tiện nghi': 8.9, 'Sự s...",nhập ngày tháng lưu trú và kiểm tra các điều ...
5,b60a7eb5,booking.com,"★★★ Golden Star Villa Hue, Huế, Việt Nam",Golden Star Villa Hue Golden Star Villa Hue \...,"{'Hiển thị bản đồ Tiện nghi': 8.8, 'Sự s...",nhập ngày tháng lưu trú và kiểm tra các điều ...
6,b3dd1d62,booking.com,"HueNam Residence, Huế, Việt Nam",HueNam Residence HueNam Residence \nVị trí xu...,"{'hiển thị bản đồ Tiện nghi': 9.5, 'Sự s...",nhập ngày tháng lưu trú và kiểm tra các điều ...
7,160a05ee,booking.com,"Khách sạn Le Domaine De Cocodo, Huế, Việt Nam",Khách sạn Le Domaine De Cocodo Khách sạn Le Do...,"{'Hiển thị bản đồ Tiện nghi': 8.8, 'Sự s...",Các loại căn hộ khác nhau có thể có chính sách...
8,a535dd90,booking.com,"★★★ Thuy Duong Boutique Hotel Hue, Huế, Việt Nam",Thuy Duong Boutique Hotel Hue Thuy Duong Bouti...,"{'hiển thị bản đồ Tiện nghi': 8.6, 'Sự s...",nhập ngày tháng lưu trú và kiểm tra các điều ...
9,667c234e,booking.com,"★★★ SALA HOTEL HUE, Huế, Việt Nam",SALA HOTEL HUE SALA HOTEL HUE \nVị trí tuyệt ...,"{'Hiển thị bản đồ Tiện nghi': 8.6, 'Sự s...",nhập ngày tháng lưu trú và kiểm tra các điều ...


In [7]:
data.to_excel('data_booking.xlsx')

In [6]:
data.to_csv('data_booking.csv')

#Apply Model to summary

In [ ]:
import torch
if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")

In [ ]:
from langdetect import detect
for index, row in data.iterrows():
    text = row['content']
    if detect(text) == "vi":
        import pandas as pd
        from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
        import torch
        tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
        model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        preprocess_text = text.strip().replace("\n", "")
        tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
        summary_ids = model.generate(
           tokenized_text,
           max_length=512,
           num_beams=4,
           repetition_penalty=2.5,
           length_penalty=1.0,
           early_stopping=True
    )
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print("\n\nSummarized: \n", output)

TypeError: ignored

In [ ]:
from langdetect import detect
for index, row in data.iterrows():
    text = row['content']
    if detect(text) == "vi":
        import pandas as pd
        from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
        import torch
        tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
        model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        preprocess_text = text1.strip().replace("\n", "")
        tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
        summary_ids = model.generate(
           tokenized_text,
           max_length=512,
           num_beams=4,
           repetition_penalty=2.5,
           length_penalty=1.0,
           early_stopping=True
    )
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print("\n\nSummarized: \n", output)



Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


# test

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")

In [ ]:
def summarization(data,tokenizer,model):
    from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
    import torch
    from langdetect import detect
    import pandas as pd
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    for val in data["content"].values:
        text=str(val)
    for index, row in data.iterrows():
        text = row['content']
        if detect(text) == "vi":
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            preprocess_text = text.strip().replace("\n", "")
            tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
            summary_ids = model.generate(
                tokenized_text,
                max_length=512,
                num_beams=8,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )
            output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            print("\n\nSummarized: \n", output)
            data.loc[index, 'summary'] = output
    return data

In [ ]:
summarization(data=data,tokenizer=tokenizer,model=model)

TypeError: ignored

In [ ]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
tokenizer.save_pretrained
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model = model.to(device)
def summarization(text):
        preprocess_text = text.strip().replace("\n", "")
        tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
        summary_ids = model.generate(
                tokenized_text,
                max_length=256,
                num_beams=4,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return output

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
tokenizer.save_pretrained('/content/drive/MyDrive/summary/tokenizer_pretrained')
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
model.save_pretrained('/content/drive/MyDrive/summary/model_pretrained')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
summarization(text="""Địa phương sẽ chi 670 tỷ đồng, trong tổng số 1.333 tỷ đồng giải phóng mặt bằng đoạn 19,5 km cao tốc Biên Hòa – Vũng Tàu qua địa phận tỉnh này.
Nội dung được thông qua tại kỳ họp HĐNĐ tỉnh Bà Rịa – Vũng Tàu khóa VII, nhiệm kỳ 2021-2026, ngày 12/4.
Động thái của tỉnh đưa ra sau khi dự án tuyến đường được Thủ tướng chấp thuận đầu tư bằng ngân sách nhà nước.
Cao tốc Biên Hòa – Vũng Tàu (giai đoạn 1) dài 53,7 km, quy mô 4-6 làn xe, tổng mức đầu tư 17.837 tỷ đồng, dự kiến khởi công đầu năm sau và hoàn thành năm 2025.""")

'Tỉnh Bà Rịa – Vũng Tàu sẽ chi 670 tỷ đồng, trong tổng số 1.333 tỷ đồng giải phóng mặt bằng đoạn 19,5 km cao tốc Biên Hòa – Vũng Tàu qua địa phận tỉnh này. Động thái của tỉnh đưa ra sau khi dự án tuyến đường được Thủ tướng chấp thuận đầu tư bằng ngân sách nhà nước.'